## Прогноз вероятности неврологических проявлений при болезни Вильсона-Коновалова. Линейная логистическая регрессия

* Импорт библиотек
* Чтение данных
* Пропуски в данных
* Группы признаков
* Отбор признаков для модели
* Оптимизация параметров и оценка качества моделей
* Выводы

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# https://stackoverflow.com/questions/11707586/python-pandas-how-to-widen-output-display-to-see-more-columns
pd.set_option('display.max_columns', 70)   # Настройка отображения данных в Jupyter notebook
pd.set_option('display.max_rows', 100)     
pd.set_option('precision', 3)
pd.set_option('display.width', 1000)
pd.set_option('display.height', 1000)
np.core.arrayprint._line_width = 100

%pylab inline
# Visualization and Graphics
%matplotlib inline
import matplotlib.pyplot as plt

# !conda install seaborn 
import seaborn as sns
plt.rcParams['figure.figsize'] = (7,7)   # (8,6)

# conda install -c conda-forge ggplot     # Suggests to down-grade matplotlib and some other packages - rejected
#!pip install ggplot                      # Installs OK
matplotlib.style.use('ggplot')            # Use ggplot style plots

### Чтение исходных данных

In [ ]:
# Чтение данных из сохранённого файла
data_dir = '../data_transforemed/'

df_ext = pd.DataFrame()
df_ext = pd.read_csv(data_dir + 'Wilson_ext.csv', sep=';', encoding='utf-8') 
print(df_ext.columns)

### Пропуски в данных
Заполним пропуски **средними значениями**. <br> Это удобно для выбранного алгоритма.

In [ ]:
# В каких колонках есть пропуски?
for c in df_ext.columns:
    number_of_na = np.sum(df_ext[c].isna())            # Needs Pandas 22.0 or higher
    if number_of_na > 0:
        print(c, number_of_na, df_ext[c].mean())

df = df_ext.fillna(df_ext.mean())                      # Заполним пропуски средними значениями

print( df_ext[['TargetHeadRelativeMax', 'BMI']].fillna(df_ext.mean()).head(8) )

### Группы признаков
Группы выделены в соответствии со смыслом данных, логикой их использования и обработки, форматом представления данных.
<br> Группы сформированы в файле `preprocess_data.ipynb`
- `target`
- `relatives`
- `sex`, `sex_cat`
- `bmi`, `bmi_scaled`
- `symptom`
- `cirrhosis`
    - `childpugh_dummy`
- `debut_age`, `debut_age_scaled`
- `debut_organ`
- `genetic`
- `genetic_dummy`
- `genetic__1`, `genetic__2`
- `genetic_risk__1`, `genetic_risk__2`
- `genetic_risk__1_scaled`, `genetic_risk__2_scaled`
- `data`
- `exclude`, `exclude_model`

<br> Вспомогательные признаки
- `num_to_scale`
- `num_scaled`
- `genetic_risk`
- `genetic_risk_scaled`

In [ ]:
# !conda install simplejson
%run data_to_json

In [ ]:
from collections import OrderedDict

In [ ]:
features_file = open(data_dir + 'features_file.json', 'r')
features = json_to_data( features_file.read() )
features_file.close()

In [ ]:
features['relatives']
features['bmi']
features['cirrhosis']
#print(features)
#type(features)

for k in features.keys():
    print(k)

In [ ]:
# Функция скопирована из `preprocess_data.ipynb`

def combine_features(features_groups,
                     exclude_features):
    '''features_groups - объединить эти группы признаков
       exclude_features  - исключить эти признаки
    '''
    result = list()
    for sublist in features_groups:
        for item in sublist:
            result.append(item)
    result = [x for x in result if x not in exclude_features]
    return(result)

## Описание эксперимента

### Название эксперимента, директория для результатов
ToDo: журнал экспериментов, отчёт о результатах.

In [ ]:
experiment_name = 'Predict_LogisticRegression-GeneticRisk__1_Scaled_2'
experiment_dir = '../experiment_dir/' + experiment_name + '/'
!md "{experiment_dir}"

### Отбор признаков для моделей

In [ ]:
# Целевой признак
target_features = features['target']       
print(target_features)
y   = np.array(df[target_features]).T[0]
y

In [ ]:
# Без генетических факторов
model_0_features = combine_features(
                       [
                        # target_features, 
                        features['relatives'],        # Макс. target у родственников
                        features['sex'],              # Пол
                        [features['bmi_scaled'][0]],  # BMI_scaled
                        features['symptom'],          # ККФ: кольцо Кайзера-Флейшера
                        [  features['cirrhosis'][0],   # 0: Cirrhosis, 1: ChildPugh, 2: Advanced
                        #   features['cirrhosis'][1]
                        #  # features['cirrhosis'][2]
                           features['childpugh_dummy'][1],
                         ],
                        features['activity'],         
                        features['debut_age_scaled'],
                        features['debut_organ'],      # Дебют, кроме неврологического
                        ],
                       features['exclude_model'])
print(model_0_features)

In [ ]:
df_ext[model_0_features].head()

In [ ]:
# С генетическими факторами

#model_genetic_features = features['genetic_risk__1_scaled']
#model_genetic_features = [features['genetic_risk__1_scaled'][0]]

#model_genetic_features = features['genetic_risk__1']
model_genetic_features = [ features['genetic_risk__1'][0] ]

print( model_genetic_features )

model_features = combine_features(
                       [  model_0_features,
                          model_genetic_features
                        ],
                       features['exclude_model'])
print(model_features)

In [ ]:
# Масштабируем генетические признаки и цирроз
# Выбор масштаба

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
scaler_st = StandardScaler()

#X   = pd.DataFrame(scaler_st.fit_transform(X), columns=X.columns)


# Cirrhosis features
df[features['cirrhosis']] = scaler_st.fit_transform(df[features['cirrhosis']])


# Genetic features
#f_val        = df.loc[:, [model_genetic_features[0]]]
f_val        = df.loc[:, model_genetic_features]
up_quantiles = f_val.quantile(0.95)
up_outliers  = (f_val > up_quantiles)
f_crop       = f_val.mask(up_outliers, up_quantiles, axis=1)

scaler.fit(f_crop)
f_scaled     = pd.DataFrame(scaler.transform(f_val))
f_scaled.columns = model_genetic_features

#scale_genetic_features = 1       # 13 признаков
scale_genetic_features = 2        # 13 признаков
#scale_genetic_features = 3       #  3 признака: DebutLiver, GenRisk__1, Sex
#scale_genetic_features = 5       #  3 признака: DebutLiver, GenRisk__1, Sex
#scale_genetic_features = 10      #  3 признака: DebutLiver, GenRisk__1, Sex
f_scaled     = f_scaled.multiply(scale_genetic_features)

#scale_2 = 10
#f_scaled.loc[:,('GenProtect__1')] = f_scaled.loc[:,('GenProtect__1')].multiply(scale_2)

print(model_genetic_features[0])

In [ ]:
model_features

In [ ]:
df.columns

In [ ]:
X = df[model_features]

X.loc[:,model_genetic_features] =  np.array(f_scaled)

X[model_genetic_features].head(4)

In [ ]:
# Без генетических факторов
# X_0 = df[model_0_features]
X_0 = X[model_0_features]

In [ ]:
#plt.hist(f_scaled.T)
#
#plt.hist(np.array(X.loc[:,(model_genetic_features)]).T[0])
#plt.xlabel(model_genetic_features[0])

In [ ]:
model_genetic_features[0]
model_genetic_features

In [ ]:
df[target_features + model_features].to_csv(experiment_dir + 'model_features.csv')
df[target_features + model_0_features].to_csv(experiment_dir + 'model_0_features.csv')

## Проведение эксперимента

### Оптимизация параметров и оценка качества моделей

In [ ]:
from sklearn.linear_model    import LinearRegression, LogisticRegression, LassoCV, Lasso

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

# param_grid = { 'C': np.logspace(-6, 2, 200) }
param_grid = { 'C': np.logspace(-6, 2, 17) }
param_grid = { 'C': np.logspace(-6, 2, 25) }
param_grid = { 'C': np.logspace(-6, 2, 33) }
param_grid = { 'C': np.logspace(-6, 2, 49) }
param_grid = { 'C': np.logspace(-6, 2, 65) }
param_grid = { 'C': np.logspace(-6, 2, 97) }
param_grid = { 'C': np.logspace(-6, 2, 129) }
#param_grid = { 'C': np.logspace(-6, 2, 193) }
#param_grid

In [ ]:
%%time

grid_searcher = GridSearchCV(                     # For model with genetic features
    estimator = LogisticRegression(random_state=42, penalty = 'l1'),
    param_grid = param_grid,
    scoring = 'roc_auc',
    cv = kfold, n_jobs=1)

grid_searcher.fit(X, y)
print('grid_searcher.fit(X, y) best score:', grid_searcher.best_score_, grid_searcher.best_params_)

In [ ]:
%%time
grid_searcher_0 = GridSearchCV(                  # For model without genetic features
    estimator=LogisticRegression(random_state=42, penalty = 'l1'),
    param_grid= param_grid,
    scoring = 'roc_auc',
    cv=kfold, n_jobs=1)

grid_searcher_0.fit(X_0, y)
print('grid_searcher_0.fit(X_0, y) best score:', grid_searcher_0.best_score_, grid_searcher_0.best_params_)

#### Зависимость ROC AUC от параметра регуляризации C = $1/\gamma$

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
results   = pd.DataFrame(grid_searcher.cv_results_)
results_0 = pd.DataFrame(grid_searcher_0.cv_results_)

#   plt.plot(results['param_C'],   results['mean_train_score'],   label= 'X:  train ROC AUC')
plt.semilogx(results['param_C'],   results['mean_test_score'],    label= 'Genetic risk used:     test data')

#   plt.plot(results_0['param_C'], results_0['mean_train_score'], label= 'X0: train ROC AUC')
plt.semilogx(results_0['param_C'], results_0['mean_test_score'],  label= 'Genetic risk ignored: test data')
plt.legend(loc='lower center');
plt.title('Logistic Regression ROC AUC') 
ymin, ymax = (0.65, 0.85)
plt.ylim( (ymin, ymax) )
xmin, xmax = (0.05, 100)
plt.xlim( (xmin, xmax) )
plt.xlabel('Regualarisation C')
plt.ylabel('ROC AUC')

#results_0   #.describe().T

#### Важность признаков у лучших моеделей

In [ ]:
def plot_importances(df, grid_searcher, param_grid, model_features, model_type_name, features_set_name):
    importances = grid_searcher.best_estimator_.coef_[0]
    X   = df[model_features]

    indices_abs = np.argsort(np.abs(importances))[::-1]    # Absolute importance
    indices_ord = np.argsort(importances)[::-1]            # rank importance

    indices = indices_abs
    model_features_xticks = [model_features[i] for i in indices]

    # Plot the feature importances
    plt.figure(figsize=(8,4))
    plt.title("Feature importances")
    plt.bar( range(X.shape[1]), np.abs(importances[indices]),
             color="r", # yerr=std[indices], align="center"
           )
    plt.xticks(range(X.shape[1]), model_features_xticks, rotation=75)
    plt.xlim([-1, X.shape[1]])
    plt.title('Важность признаков лучшей модели ' + model_type_name + '\n' + features_set_name)
    plt.show()

plot_importances(df, grid_searcher, param_grid, model_features,     "LogisticRegression", "с генетическими факторами риска")
plot_importances(df, grid_searcher_0, param_grid, model_0_features, "LogisticRegression",  "без генетических факторов риска")

In [ ]:
def report_importances(grid_searcher, param_grid, model_features, model_type_name, features_set_name):
    
    # -----------------------------------------------------
    # Print featires of the best model
    # -----------------------------------------------------
    importances = grid_searcher.best_estimator_.coef_[0]
    X   = df[model_features]

    indices_abs = np.argsort(np.abs(importances))[::-1]    # Absolute importance
    indices_ord = np.argsort(importances)[::-1]            # rank importance

    #print("Feature importance:")
    #indices = indices_abs
    #for f in range(len(model_features)):                 #  range(X.shape[1]):
    #    if np.abs(importances[indices[f]])>0:
    #        print("%d. %-20s \t%f" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

    print("\nВеса признаков для модели " + model_type_name + "\n" + features_set_name)
    
    indices = indices_ord            # Вывести отсчёт, сортируя признаки с учётом знака
    indices = indices_abs            # Вывести отсчёт, сортируя признаки по абсолютной величине
    
    important_features = []
    for f in range(len(model_features)):                 #  range(X.shape[1]):
        if np.abs(importances[indices[f]])>0:
            important_features.append(X.columns[indices[f]])
            print("%d. %-20s \t%f" % (f + 1, X.columns[indices[f]], importances[indices[f]]))
    
    return(important_features)

important_features   = report_importances(grid_searcher,   param_grid, model_features,   
                                          "LogisticRegression", "с генетическими факторами риска")

important_features_0 = report_importances(grid_searcher_0, param_grid, model_0_features, 
                                          "LogisticRegression",  "без генетических факторов риска")

print('\n', important_features)
print('\n', important_features_0)

#### Зависимость коэффцицентов модели от параметра регуляризации C = $1/\gamma$

Оптимизируем $C \cdot \sum  LogLoss(i) + \sum |w_i|$. 
<br> - Большая $C$: доминирует $\sum LogLoss$ - слабая регуляризация.
<br> - Малая $C$: доминирует $\sum |w_i|$ - сильная регуляризация.


In [ ]:
def plot_importances_path(grid_searcher, param_grid, model_features, model_type_name, features_set_name):
    importances = grid_searcher.best_estimator_.coef_[0]
    X   = df[model_features]

    indices_abs = np.argsort(np.abs(importances))[::-1]    # Absolute importance
    indices_ord = np.argsort(importances)[::-1]            # rank importance

    # -----------------------------------------------------
    # Generate data
    # -----------------------------------------------------
    coefs   = []
    estimator=LogisticRegression(random_state=42, penalty = 'l1')
    for C in param_grid["C"]:
        params = {"C":C}
        estimator.set_params(**params)
        estimator.fit(X, y)
        coef_ = estimator.coef_[0]
        coefs.append(coef_)

    # -----------------------------------------------------
    # Graphics
    # -----------------------------------------------------
    ax = plt.gca()
    indices = indices_ord
    for f in range(len(model_features)):
        i = indices[f]
        feature = model_features[i]
        feature_data = np.array(coefs).T.tolist()[i]
        if np.abs(importances[indices[f]])>0:
            ax.plot(param_grid["C"], feature_data, label=feature)
        else:
            ax.plot(param_grid["C"], feature_data)

    ax.set_prop_cycle(cycler('color', 
                          ['b', 'r', 'g', 'c', 'k', 'y', 'm']))

    ax.set_xscale('log')
    plt.xlabel('Коэффициент регуляризации C')
    plt.ylabel('Коэффциенты модели')
    plt.title('Важность признаков\nмоделей ' + model_type_name + '\n' + features_set_name)
    
    ax.set_xlim(left=0.01, right=grid_searcher.best_params_["C"])   # Диапазон по оси x   100)
    ax.set_ylim(bottom=-3, top=2)                                   # Диапазон по оси y

    # Put a legend to the right: https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot
    box = ax.get_position()      # print("box: ", box)
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height * 0.6]) # Shrink current axis to 80% (x) and 60% (y).
    ax.legend()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.50))

    ax.minorticks_on             # Grid at minor ticks
    ax.grid(True, which='both')

    plt.show();
# -----------------------------------------------------------------

plot_importances_path(grid_searcher,   param_grid, model_features,   "LogisticRegression", "с генетическими факторами риска")
plot_importances_path(grid_searcher_0, param_grid, model_0_features, "LogisticRegression",  "без генетических факторов риска")

### Leave-one-out cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict, LeaveOneOut, KFold, ShuffleSplit
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc

In [ ]:
name  = "Genetic risk factors included."
clf = LogisticRegression(random_state=42, penalty = 'l1', C=grid_searcher.best_params_["C"] ) #C=0.38311868495572848) 
cv_predictions = cross_val_predict(clf, X, y,
                                       method='predict_proba',
                                       cv=LeaveOneOut()
                                       #cv = kfold
                                  ) #  cv=10)


name_0  = "Genetic risk factors ignored."
clf_0 = LogisticRegression(random_state=42, penalty = 'l1', C=grid_searcher_0.best_params_["C"])
cv_predictions_0 = cross_val_predict(clf_0, X_0, y,
                                       method='predict_proba',
                                       cv=LeaveOneOut()
                                       #cv = kfold
                                    ) #  cv=10)

fpr,   tpr,   thresh = roc_curve(y, cv_predictions[: ,1],   drop_intermediate=False)
fpr_0, tpr_0, thresh = roc_curve(y, cv_predictions_0[: ,1], drop_intermediate=False)
    
figure(figsize=(8,8))
plt.plot([0,1],[0,1],color='darkgray')
plt.plot([0,0],[0,1],color='darkgray')
plt.plot([0,1],[0,0],color='darkgray')
plt.plot([0,1],[1,1],color='darkgray')
plt.plot([1,1],[0,1],color='darkgray')

plt.plot(fpr,   tpr,   label= name  
                              + ' ROC AUC=' + str( round(auc(fpr,   tpr  )*1000)/1000. ) )
plt.plot(fpr_0, tpr_0, label= name_0
                              + '  ROC AUC=' + str( round(auc(fpr_0, tpr_0)*1000)/1000. ) )

xlabel('False Positive Rate')
ylabel('True Positive Rate')
title('ROC кривые для Logistic Regression')
legend(loc='lower center');

plt.savefig(experiment_dir + 'LogRegression.png', bbox_inches='tight')

### Сравнение предсказаний

In [ ]:
#pred_sorted   = -numpy.sort(-cv_predictions[:,1])
#pred_0_sorted = -numpy.sort(-cv_predictions_0[:,1])
pred_diff = cv_predictions[:,1] - cv_predictions_0[:,1]

pred_array = np.array([
    #list(range(pred_length)),
    cv_predictions_0[:,1],
    cv_predictions[:,1],
    pred_diff[:]
    ]).transpose()

print(len(pred_array))

pred_array_sorted_0 = pred_array[pred_array[:,0].argsort()]
pred_array_sorted   = pred_array[pred_array[:,1].argsort()]

#plt.scatter( x=range(len(pred_array_sorted_0)), y=pred_array_sorted_0[:,0])
#plt.scatter( x=range(len(pred_array_sorted_0)), y=pred_array_sorted_0[:,1])

In [ ]:
plt.figure(figsize=(6, 4))
#plt.subplots(1,2, figsize=(12, 4))
#plt.subplots(2,1, figsize=(6, 8))

# plt.subplot(1,2,1)
plt.scatter( x=range(len(pred_array_sorted_0)), y=pred_array_sorted_0[:,1], color = 'tab:blue', label='Genetic risk factors',    s=15)
plt.scatter( x=range(len(pred_array_sorted_0)), y=pred_array_sorted_0[:,0], color = 'tab:red',  label='No genetic risk factors', s=15)
#
#plt.scatter(range(pred_length), pred_sorted,   color = 'tab:blue',  label='Genetic risk factors', s=15)
#plt.scatter(range(pred_length), pred_0_sorted, color = 'tab:red', label='No genetic risk factors', s=15)
plt.legend()
plt.title("Оценки вероятности: P,  P0")
plt.xlabel("Rank,  Rank0\n")
plt.ylabel("P,  P0")

#plt.subplot(1,2,2)
#plt.axhline(y=0, color='darkgrey', linestyle='-',linewidth=1)
#
#p = plt.scatter(cv_predictions_0[:,1],               # pred_0_sorted, 
#                pred_diff, s=15)
#plt.title("Разность оценок вероятности: P - P0")
#plt.xlabel("P0")                                  # plt.xlabel("P0")
#plt.ylabel("P - P0")
#p.set_sizes(3)

### Кластеризация данных, взвешенных с помощью важности признаков

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Features scaled with importances
importances   = grid_searcher.best_estimator_.coef_[0]
importances_0 = grid_searcher_0.best_estimator_.coef_[0]

X_scaled = np.array(X).dot(np.diag(importances))
X_0_scaled = np.array(X_0).dot(np.diag(importances_0))

In [ ]:
# Масштабируем выборку с помощью StandardScaler с параметрами по умолчанию.
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_0_scaled = scaler.fit_transform(X_0)

In [ ]:
# Понижаем размерность с помощью PCA, оставляя столько компонент, сколько нужно, чтобы 
# объяснить как минимум 90% дисперсии исходных (отмасштабированных) данных. 
# Используем отмасштабированную выборку и фиксируем random_state (константа RANDOM_STATE).

RANDOM_STATE=1543

pca     = PCA(n_components=0.9, random_state=RANDOM_STATE).fit(X_scaled)
X_pca   = pca.transform(X_scaled)

pca_0   = PCA(n_components=0.9, random_state=RANDOM_STATE).fit(X_0_scaled)
X_0_pca = pca_0.transform(X_0_scaled)

In [ ]:
plt.subplots(1,2, figsize=(12, 4))

plt.subplot(1,2,1)
plt.scatter(X_0_pca[:,0], X_0_pca[:,1], c=y, alpha=0.15, cmap = 'jet')
plt.title("Группировка без генетических признаков")

plt.subplot(1,2,2)
#X_plt_dat = np.round( X_pca.transpose(), 2)
plt.scatter(X_pca[:,0],   X_pca[:,1], c=y, alpha=0.15, cmap = 'jet')
plt.title("Группировка с использованием \n генетических признаков")


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_classes = 2
kmeans = KMeans(n_clusters=n_classes, n_init=100, 
                random_state=RANDOM_STATE, n_jobs=1)
kmeans.fit(X_pca)
cluster_labels = kmeans.labels_

In [ ]:
#n_classes = 4
#RANDOM_STATE=123
#kmeans_0 = KMeans(n_clusters=n_classes, n_init=100, 
#                  init = np.array([[-0.7,  0.6],  [-0.5,-0.2], [0.9,0.85], [1,0.05]], np.float64),
#                  random_state=RANDOM_STATE, n_jobs=1)

n_classes = 2
kmeans_0 = KMeans(n_clusters=n_classes, n_init=100, 
                random_state=RANDOM_STATE, n_jobs=1)

kmeans_0.fit(X_0_pca[:,0:2])
cluster_0_labels = kmeans_0.labels_

In [ ]:
plt.subplots(3,2, figsize=(10, 11))

plt.subplot(3,2,1)
plt.scatter(X_0_pca[:,0], X_0_pca[:,1], c=y, alpha=0.15, cmap = 'jet')
plt.title("Группировка без \n генетических признаков")

plt.subplot(3,2,2)
#X_plt_dat = np.round( X_pca.transpose(), 2)
plt.scatter(X_pca[:,0],   X_pca[:,1], c=y, alpha=0.15, cmap = 'jet')
plt.title("Группировка с использованием \n генетических признаков")


plt.subplot(3,2,3)
plt.scatter(X_0_pca[:, 0], X_0_pca[:, 1], c=-cluster_0_labels, s=20,  cmap='jet');
plt.title("Поставим вручную метки классов")

plt.subplot(3,2,4)
plt.scatter(X_pca[:, 0], X_pca[:, 1],     c=-cluster_labels,   s=20,  cmap='jet');
plt.title("Поставим вручную метки классов")

plt.subplot(3,2,5)
plt.scatter(-X_0['DebutLiver'], X_0['TargetHeadRelativeMax'], c=y, s=20, alpha=0.15,  cmap='jet');
plt.title("x: -DebutLiver,   y: TargetHeadRelativeMax")

plt.subplot(3,2,6)
plt.scatter(-X['DebutLiver'],     X[model_genetic_features[0]], c=y, s=20, alpha=0.15, cmap='jet');
plt.title("x: -DebutLiver,   y: " + model_genetic_features[0])


plt.savefig(experiment_dir + 'LogRegression_Clustering.png', bbox_inches='tight')

In [ ]:
exclude_features = features['exclude']

df_pca = df[combine_features([ target_features, important_features ], exclude_features)]
df_pca['cluster_labels'] = cluster_labels

outfile = open(experiment_dir + './Wilson_pca.csv', 'w')
df_pca.to_csv(outfile, sep=';', index=False, encoding='utf-8', chunksize=1)
outfile.close()

In [ ]:
df_pca_0 = df[combine_features([ target_features, important_features_0 ], exclude_features)]
df_pca_0['cluster_0_labels'] = cluster_0_labels
df_pca_0

outfile = open(experiment_dir + './Wilson_0_pca.csv', 'w')
df_pca.to_csv(outfile, sep=';', index=False, encoding='utf-8', chunksize=1)
outfile.close()

## Bootstrap доверительные интервалы

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


def compare_bootstrap_predictions(df, model_0_features, model_features, target_features,
                                  clf_0, clf, 
                                  cat_features=[], n_samples=100, n_splits=10):
    
    X_0 = df[model_0_features]
    X   = df[model_features]
    y   = np.array(df[target_features]).T[0]
    
    sample_size = X.shape[0]
    predictions   = np.zeros((n_samples, sample_size))
    predictions_0 = np.zeros((n_samples, sample_size))
    i_split = 0
    for train_indices, test_indices in KFold(n_splits=n_splits, 
                                             shuffle=True, random_state=1543).split(X):
        i_split = i_split+1;
        #print('i_split:',i_split)
        
        boot_test_X_0 = X_0.loc[test_indices]
        boot_test_X   = X.loc[  test_indices]

        # Таблица индексов для всех bootstrap выборок
        boot_train_indices = np.random.choice(train_indices, size=(n_samples, train_indices.shape[0]))
        
        for sample_id in range(n_samples):
            boot_train_X_0 = X_0.loc[boot_train_indices[sample_id]]
            boot_train_X   = X.loc[  boot_train_indices[sample_id]]
            boot_train_y   = y[boot_train_indices[sample_id]]
            
            if type(clf).__name__ == 'CatBoostClassifier':
                clf_0.fit(boot_train_X_0, boot_train_y, cat_features=cat_features, logging_level='Silent')
                clf.fit(  boot_train_X,   boot_train_y, cat_features=cat_features, logging_level='Silent')
            else:
                clf_0.fit(boot_train_X_0, boot_train_y)
                clf.fit(  boot_train_X,   boot_train_y)

            predictions_0[sample_id, test_indices] = clf_0.predict_proba(boot_test_X_0)[:,1]
            predictions[  sample_id, test_indices] = clf.predict_proba(  boot_test_X)[:,1]

    return (predictions_0, predictions)


def make_bootstrap_predictions(X, y, clf, cat_features=[], n_samples=100, n_splits=10):
    sample_size = X.shape[0]
    predictions = np.zeros((n_samples, sample_size))
    i_split = 0
    for train_indices, test_indices in KFold(n_splits=n_splits).split(X):
        i_split = i_split+1;
        #print('i_split:',i_split)
        
        boot_test_X = X.loc[test_indices]

        # Таблица индексов для всех bootstrap выборок
        boot_train_indices = np.random.choice(train_indices, size=(n_samples, train_indices.shape[0]))

        for sample_id in range(n_samples):
            boot_train_X = X.loc[boot_train_indices[sample_id]]
            boot_train_y = y[boot_train_indices[sample_id]]
            if type(clf).__name__ == 'CatBoostClassifier':
                clf.fit(boot_train_X, boot_train_y, cat_features=cat_features, logging_level='Silent')
                predictions[sample_id, test_indices] = clf.predict_proba(boot_test_X)[:,1]
                # predictions[sample_id, test_indices] = clf.predict(boot_test_X)
            else:
                clf.fit(boot_train_X, boot_train_y)
                predictions[sample_id, test_indices] = clf.predict_proba(boot_test_X)[:,1]
            
    return predictions


def count_roc_aucs(y, predictions):
    samples_count = predictions.shape[0]
    result = np.zeros(samples_count)
    for i in range(samples_count):
        result[i] = roc_auc_score(y, predictions[i])
    return result

In [ ]:
%%time

predictions_0, predictions = compare_bootstrap_predictions(
    df, model_0_features, model_features, target_features, 
    clf_0, clf, cat_features=[], n_samples=1000, n_splits=10)

In [ ]:
rocs_0 = count_roc_aucs(y, predictions_0)
rocs   = count_roc_aucs(y, predictions)
rocs_diff = [rocs[i] - rocs_0[i] for i in range(len(rocs))]

In [ ]:
bins = np.linspace(-0.03, 0.14, 1+np.round((0.14 - (-0.03))/0.005))
bins
#1+np.round((0.14 - (-0.03))/0.01)

In [ ]:
plt.figure(figsize=(12, 4))

p121 = plt.subplot(121)
bins = np.linspace(0.6, 0.9, 21)
n, b, p    = pyplot.hist(rocs,   bins, normed = True, alpha=0.5, label='С генетическими признаками')
n0, b0, p0 = pyplot.hist(rocs_0, bins, normed = True, alpha=0.5, label='Без генетических признаков')
pyplot.legend(loc='upper left')
#plt.title('Статистическая (bootstrap) оценка ROC AUC для \n моделей с генетичесими признаками и без')
p121.set_title('Статистическая (bootstrap) оценка ROC AUC для \n моделей с генетичесими признаками и без',fontsize=12)
pyplot.xlabel('ROC AUC')
pyplot.ylabel('Частота  ROC AUC')
axes = plt.gca()
axes.set_ylim([0,20])

p122 = plt.subplot(122)
#bins = np.linspace(-0.02, 0.14, 21)
xmin = -0.02
xmax = 0.13
step = 0.01
bins = np.linspace(xmin, xmax, 1+np.round((xmax - xmin)/step))

plt.hist(rocs_diff, bins, alpha = 0.65, normed=1)
#plt.title('Статистическая оценка разности ROC AUC для \n моделей с генетичесими признаками и без')
p122.set_title('Bootsrap-разности ROC AUC для \n моделей с генетичесими признаками и без',fontsize=12)
plt.ylabel('Частота  разности')
plt.xlabel('Разность ROC AUC')
axes = plt.gca()
axes.set_ylim([0,20])

#plt.show()

plt.savefig(experiment_dir + 'ROC_AUC_diff.png', bbox_inches='tight')

In [ ]:
plt.hist(rocs_diff, alpha = 0.75, normed=1)
plt.title('Статистическая оценка разности ROC AUC для \n моделей с генетичесими признаками и без')
plt.ylabel('Частота')
plt.xlabel('Разность ROC AUC')
pyplot.show()

In [ ]:
clf

In [ ]:
clf_0

### Доверительный интервал для tpr

In [ ]:
predictions.shape
#np.around(fpr, 5)

predictions.shape[0]

In [ ]:
def get_tpr_bounds(y, predictions):
    
    result = []
    roc_dict = dict()
    for i in range(predictions.shape[0]):
        fpr, tpr, thresh = roc_curve(y, predictions[i], drop_intermediate=False)
        fpr = np.around(fpr, 5)
        tpr = np.around(tpr, 5)
        #print(i)
        for j in range(len(tpr)):
            #if(fpr[j]==0):
            #    print(fpr[j], tpr[j])
            if fpr[j] in roc_dict.keys():
                roc_dict[fpr[j]].append(tpr[j])
            else:
                roc_dict[fpr[j]] = [tpr[j]]

    for f in sorted(roc_dict):
        t = roc_dict[f]
        tpr_low  = np.percentile(t, q=2.5, axis=0)
        tpr_med  = np.percentile(t, q=50, axis=0)
        tpr_up   = np.percentile(t, q=97.5, axis=0)
        result.append([f, tpr_low, tpr_med, tpr_up])  
    
    result = np.array(result)
    return(result, roc_dict)

roc_0_bounds, roc_0_dict = get_tpr_bounds(y, predictions_0)
roc_bounds,   roc_dict   = get_tpr_bounds(y, predictions)

In [ ]:
figure(figsize=(5,5))
plt.plot([0,1],[0,1],color='darkgray')
plt.plot([0,0],[0,1],color='darkgray')
plt.plot([0,1],[0,0],color='darkgray')
plt.plot([0,1],[1,1],color='darkgray')
plt.plot([1,1],[0,1],color='darkgray')


plt.fill_between(roc_bounds[:,0],   roc_bounds[:,1],   roc_bounds[:,3],   color='red', alpha=0.2)

plt.fill_between(roc_0_bounds[:,0], roc_0_bounds[:,1], roc_0_bounds[:,3], color='navy', alpha=0.2)


plt.plot(roc_bounds[:,0], roc_bounds[:,1], 
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'red', alpha = 0.12
        )

plt.plot(roc_bounds[:,0], roc_bounds[:,2], 
         label='С генетическими признаками',
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'red', alpha = 0.65
        )

plt.plot(roc_bounds[:,0], roc_bounds[:,3], 
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'red', alpha = 0.12
        )


plt.plot(roc_0_bounds[:,0], roc_0_bounds[:,1], 
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'navy', alpha = 0.12
        )

plt.plot(roc_0_bounds[:,0], roc_0_bounds[:,2], 
         label='Без генетических признаков',
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'navy', alpha = 0.75
        )

plt.plot(roc_0_bounds[:,0], roc_0_bounds[:,3], 
         #label= name + ' ROC AUC=' + str( round(auc(fpr__0975,   tpr__0975  )*1000)/1000. )
         color = 'navy', alpha = 0.12
        )

plt.title('Доверительный коридор 95% для ROC кривых')
plt.legend(loc=(0.25, 0.07))  # 'lower right')


plt.savefig(experiment_dir + 'ROC_corridor.png', bbox_inches='tight')

### Выводы
* Комбинированный генетический фактор позволил уточнить ROC AUC: с 0.729 до 0.774. <br> Статистическая достоверность не оценивалась.
* При малых значениях параметра регяляризации $C = 1/\gamma$, лучшее ROC AUC -- у предсказателя, использующего генетические факторы риска, а при больших сравнение неоднозначно: лучше то один, то другой предсказатель.
* Признаки моделей LogisticRegression с наибольшей ROC AUC:
    * с использованием генетических факторов риска: <br> GenRisk\_\_2 (0.43), DebutAge_scaled (0.05), Activity (-0.1), Sex (-0.25),  DebutLiver (-1.41)
    * без использования генетических факторов риска: <br>  TargetHeadRelativeMax (0.99), KKF (0.89), Cirrhosis (0.27), DebutAge_scaled (0.13), Sex (-0.45), Activity (-0.49), DebutLiver (-1.55).
* Данные модели с генетическими факторами риска можно  кластеризовать с помощью двух признаков: DebutLiver, GenRisk\_\_2. -- оценить качество кластеризации

### Особенности вычисления
* Развитие цирроза описано с помощью признака Cirrhosis. 
* Оптимальные параметры сосчитаны с помощью 10-fold cross validation. <br> А измерение качества: leave-one-out cross validation.


In [ ]:
df[features['cirrhosis']].corr()

In [ ]:
df[features['cirrhosis']].head(10)

In [ ]:
df[features['cirrhosis']].head(10)

In [ ]:
#df['ChildPugh'], df['Cirrhosis']
#y
#
plt.scatter(df_ext['ChildPugh'][y==0] + np.random.uniform(-0.1, 0.1, size=(51)),
            df_ext['Cirrhosis'][y==0] + np.random.uniform(-0.1, 0.1, size=(51)), c='b', alpha = 0.15)
plt.scatter(df_ext['ChildPugh'][y==1] + np.random.uniform(-0.1, 0.1, size=(33)),
            df_ext['Cirrhosis'][y==1] + np.random.uniform(-0.1, 0.1, size=(33)), c='r',  alpha = 0.15)

In [ ]:
cirrhosis_count = {}
for i in df_ext['ChildPugh'].unique():
    cirrhosis_count[i] = {}
    for j in df_ext['Cirrhosis'].unique():
        cirrhosis_count[i][j] = (len(df_ext[np.logical_and(np.logical_and(df_ext['ChildPugh'] == i,
                                                               df_ext['Cirrhosis'] == j),
                                                y == 0)]),
                              len(df_ext[np.logical_and(np.logical_and(df_ext['ChildPugh'] == i,
                                                               df_ext['Cirrhosis'] == j),
                                                y == 1)]))
    

In [ ]:
cirrhosis_count